### boor-system

In [2]:
import sys
sys.path.append('/Users/baza/dev/python-boorstat/')

# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# import plotly.plotly as py
# import plotly.graph_objs as go

import boorstat

### boor-title

Russian Writers CSV

### boor-short-content

[empty]

### boor-full-content

In [12]:
import requests
from bs4 import BeautifulSoup

WIKI_RUS_WRITERS_URL = 'https://en.wikipedia.org/wiki/List_of_Russian-language_writers'

soup = BeautifulSoup(requests.get(WIKI_RUS_WRITERS_URL).content, 'lxml')

In [51]:
import re

all_lis = soup.find('div', id='mw-content-text').find_all('li')
print(len(all_lis))
lis = [li.text for li in all_lis if re.search(r'> \(', str(li))]

def extract_fields(li):
    name = li.partition('(')[0].strip()

    years = re.search(r'\((.+?\))', li).group(0)
    years = re.findall(r'[0-9]{4}', years)

    birth, death = '', ''
    if len(years) == 1:
        birth = years[0]
    elif len(years) == 2:
        birth, death = years
        
    descr = li.partition(')')[2]
    descr_parts = re.split(r', (?=[A-Z])', descr)
    descr = descr_parts[0].strip().strip(',')

    works = ''
    if len(descr_parts) > 1:
        works = ', '.join(descr_parts[1:])

    print(name, birth, death, descr, works)

extract_fields(lis[55])

1437
Jurgis Baltrušaitis 1873 1944  poet and translator The Pendulum


In [24]:
dir(lis[50])

['HTML_FORMATTERS',
 'XML_FORMATTERS',
 '__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_attr_value_as_string',
 '_attribute_checker',
 '_find_all',
 '_find_one',
 '_formatter_for_name',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descendant',
 '_select_debug',
 '_selector_combinators',
 '_should_pretty_print',
 '_tag_name_matches_and',
 'append',
 'attribselect_re',
 'attrs',
 'can_be_empty_element',
 'childGenerator',
 'children',
 'clear',
 'contents',
 'decode',
 'decode_contents',
 'decompose',
 'descendants',


There are more than 25 known Pushkin's duels.<br/>
Let's make CSV containing these duels details.<br/>
Here its full content:

In [14]:
import requests

DUELS_CSV_URL = 'https://raw.githubusercontent.com/boorstat/boorstat-files/master/lit/pushkin/duels.csv'

print(requests.get(DUELS_CSV_URL).text)

year,opponent_name,opponent_descr,cause,pushkin_shot,opponent_shot
1816,Paul Hannibal,uncle,during a ball Paul lugged away Pushkin’s girlfriend miss Loshakova,0,0
1817,Pyotr Kaverin,friend,Kaverin’s facetious poem,0,0
1819,Kondratiy Ryleev,poet,Ryleev told a joke about Pushkin at a high society gathering,0,0
1819,Wilhelm Kiichelbecker,friend,funny verses about Küchelbecker,0,1
1819,Modest Korf,Ministry of justice worker,Pushkin’s drunk manservant pestered Korf’s servant who finally beat Pushkin’s servant up,0,0
1819,Denisevich,Major,Pushkin behaved provocatively in theater: he yelled at actors,0,0
1820,Orlov Fedor and Alexey Alexeev,,Orlov and Alexeev reprimanded Pushkin for being drunk and trying to play pool,0,0
1821,Deguilly,French military officer,a quarrel under unclear circumstances,0,0
1822,Semyon Starov,lieutenant colonel,a conflict occurred because of a restaurant orchestra at a casino where both indulged in gambling,1,1
1822,Ivan Lanov,65-year-old state councilor,a quarrel du

<a href="https://raw.githubusercontent.com/boorstat/boorstat-files/master/lit/pushkin/duels.csv">Pushkin's duels csv</a><br/>

Source for this data:<br/>
https://rinatim.com/2016/12/03/alexander-pushkins-duels/<br/>
http://d-push.net<br/>
https://ru.wikipedia.org/wiki/Пушкин,_Александр_Сергеевич<br/>

And now let's try to use it and plot some visual representation of Pushkin's duels into real shots conversion.<br/>
Starting with imports:

In [3]:
from io import StringIO

import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go

Getting csv as pandas dataframe:

In [22]:
df = pd.DataFrame.from_csv(StringIO(requests.get(DUELS_CSV_URL).text), index_col=None)
df.head()

,year,opponent_name,opponent_descr,cause,pushkin_shot,opponent_shot
0,1816,Paul Hannibal,uncle,during a ball Paul lugged away Pushkin’s girlf...,0,0
1,1817,Pyotr Kaverin,friend,Kaverin’s facetious poem,0,0
2,1819,Kondratiy Ryleev,poet,Ryleev told a joke about Pushkin at a high soc...,0,0
3,1819,Wilhelm Kiichelbecker,friend,funny verses about Küchelbecker,0,1
4,1819,Modest Korf,Ministry of justice worker,Pushkin’s drunk manservant pestered Korf’s ser...,0,0


Looks very exciting as for me:)<br/>
Time to plot Pushkin's Duels Histogram.

In [38]:
life_years = {'start': 1799, 'end': 1837, 'size': 1}

nobody_shot_data = go.Histogram(
    x=df[(df['pushkin_shot'] == 0) & (df['opponent_shot'] == 0)]['year'],
    xbins=life_years,
    name='Nobody'
)

only_opponent_shot_data = go.Histogram(
    x=df[(df['pushkin_shot'] == 0) & (df['opponent_shot'] == 1)]['year'],
    xbins=life_years,
    name='Only opponent'
)

both_shot_data = go.Histogram(
    x=df[(df['pushkin_shot'] == 1) & (df['opponent_shot'] == 1)]['year'],
    xbins=life_years,
    name='Both shot'
)

data = [only_opponent_shot_data, nobody_shot_data, both_shot_data]
layout = go.Layout(barmode='stack', title="Pushkin's Duels Histogram")
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='pushkin-duels')